# Computation of velocity and acceleration 

- [ ] velocity
- [ ] acceleration


In [2]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd
import dask.array as da

#import cartopy.crs as ccrs
#import cartopy.feature as cfeature
#import geopandas as gpd
from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt


import drifters.utils as ut
import pynsitu as pin

from dask.delayed import delayed

In [3]:
from dask.distributed import Client

if True:
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(cores=4, processes=4, walltime='04:00:00')
    w = cluster.scale(jobs=5)
else:
    from dask.distributed import LocalCluster
    cluster = LocalCluster()
    
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.254:8787/status,
Dashboard: http://10.148.0.254:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.254:37308,Workers: 0
Dashboard: http://10.148.0.254:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
# load data

col_sel = ['time', 'id','lon', 'lat', 've', 'vn', 'typebuoy', 'gap',
           'deploy_date', 'deploy_lat','deploy_lon', 'end_date', 'end_lat', 'end_lon', 
           'drogue_lost_date','typedeath', 'lon360', 'err_lat',
           'err_lon', 'err_ve', 'err_vn']

bandwidth = 0.1

df_argos = ut.load_gdp("argos", suffix=f"_filtered_b{bandwidth}")
df_argos = df_argos.where(df_argos!=-1.e+34).reset_index().rename(columns=dict(index="id"))[col_sel].persist()

df_gps = ut.load_gdp("gps", suffix=f"_filtered_b{bandwidth}")
df_gps = (df_gps.where(df_gps!=-1.e+34).reset_index().rename(columns=dict(index="id")))[col_sel].persist() # ATTENTION : replace missing value -1.e+34 by Nan

In [4]:
df_gps

,time,id,lon,lat,ve,vn,typebuoy,gap,deploy_date,deploy_lat,deploy_lon,end_date,end_lat,end_lon,drogue_lost_date,typedeath,lon360,err_lat,err_lon,err_ve,err_vn
npartitions=176,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float32,float32,float32,float32,object,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## Computation : 
Derivation of velocities and acceleration :
- ve, vn already computed via LOWESS METHOD
- vex, vey centered derivation using projection of lon, lat
- aex, any double centered derivation using projection of lon, lat
- ae, an centered derivation of ve, vn

In [20]:
def c_vel_acc(df):
    _geo = pin.geo.GeoAccessor(df)
    _geo.compute_velocities(time="time", centered=True, names=('vex', 'vny','vxy'), inplace=True)
    print(df.columns)
    _geo.compute_accelerations(time='time', names=('ae','an','aen'), from_ = ('velocities','ve','vn'), centered_velocity=True, inplace=True )
    print(df.columns)
    _geo.compute_accelerations(time='time', names=('aex','any', 'axy'), from_ = ('lonlat','lon','lat'), inplace=True )
    print(df.columns)
    _geo.compute_velocities(time="time", distance='diff', centered=True, names=('vex_diff', 'vny_diff','vxy_diff'), inplace=True)
    print(df.columns)
    return _geo._obj

In [24]:
df_gps_out = df_gps.groupby("id").apply(c_vel_acc).persist()
df_argos_out = df_argos.groupby("id").apply(c_vel_acc).persist()

Index(['time', 'id', 'lon', 'lat', 've', 'vn', 'typebuoy', 'gap',
       'deploy_date', 'deploy_lat', 'deploy_lon', 'end_date', 'end_lat',
       'end_lon', 'drogue_lost_date', 'typedeath', 'lon360', 'err_lat',
       'err_lon', 'err_ve', 'err_vn', 'x', 'y', 'vex', 'vny', 'vxy'],
      dtype='object')
Index(['time', 'id', 'lon', 'lat', 've', 'vn', 'typebuoy', 'gap',
       'deploy_date', 'deploy_lat', 'deploy_lon', 'end_date', 'end_lat',
       'end_lon', 'drogue_lost_date', 'typedeath', 'lon360', 'err_lat',
       'err_lon', 'err_ve', 'err_vn', 'x', 'y', 'vex', 'vny', 'vxy', 'ae',
       'an', 'aen'],
      dtype='object')
Index(['time', 'id', 'lon', 'lat', 've', 'vn', 'typebuoy', 'gap',
       'deploy_date', 'deploy_lat', 'deploy_lon', 'end_date', 'end_lat',
       'end_lon', 'drogue_lost_date', 'typedeath', 'lon360', 'err_lat',
       'err_lon', 'err_ve', 'err_vn', 'x', 'y', 'vex', 'vny', 'vxy', 'ae',
       'an', 'aen', 'aex', 'any', 'axy'],
      dtype='object')
Index(['time', 'id

In [22]:
df_gps_out

,time,id,lon,lat,ve,vn,typebuoy,gap,deploy_date,deploy_lat,deploy_lon,end_date,end_lat,end_lon,drogue_lost_date,typedeath,lon360,err_lat,err_lon,err_ve,err_vn,x,y,vex,vny,vxy,ae,an,aen,aex,any,axy,vex_diff,vny_diff,vxy_diff
npartitions=176,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float32,float32,float32,float32,object,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Store in parquet

In [25]:
root_dir = "/home1/datawork/mdemol/GDP"
parquet_argos = os.path.join(root_dir, "argos_av_time.parquet")
parquet_gps = os.path.join(root_dir, "gps_av_time.parquet")

In [26]:
df_load_gps = (df_gps_out.set_index('time').repartition(partition_size="100MB").persist())
df_load_argos = (df_argos_out.set_index('time').repartition(partition_size="100MB").persist())

In [27]:
df_load_gps.to_parquet(parquet_gps, engine='pyarrow') # alternative: fastparquet pyarrow
df_load_argos.to_parquet(parquet_argos, engine='pyarrow') # alternative: fastparquet

In [26]:
df_load_gps

,id,lon,lat,ve,vn,typebuoy,gap,deploy_date,deploy_lat,deploy_lon,end_date,end_lat,end_lon,drogue_lost_date,typedeath,lon360,err_lat,err_lon,err_ve,err_vn
npartitions=88,,,,,,,,,,,,,,,,,,,,
2001-07-01 02:00:00,int64,float32,float32,float32,float32,object,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float64,datetime64[ns],float64,float32,float32,float32,float32,float32
2011-10-05 19:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-21 04:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-06 03:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Open test

In [28]:
df_gps_reload = dd.read_parquet(parquet_gps).persist()

In [29]:
df_gps_reload['ve'].compute()

time
2001-07-01 02:00:00    0.6460
2001-07-01 03:00:00    0.6466
2001-07-01 04:00:00    0.6440
2001-07-01 05:00:00    0.6368
2001-07-01 06:00:00    0.6296
                        ...  
2020-04-06 03:00:00    0.0721
2020-04-06 03:00:00   -0.0199
2020-04-06 03:00:00   -0.0878
2020-04-06 03:00:00    0.0206
2020-04-06 03:00:00    0.0386
Name: ve, Length: 41635507, dtype: float32

In [30]:
df_gps_reload.columns

Index(['id', 'lon', 'lat', 've', 'vn', 'typebuoy', 'gap', 'deploy_date',
       'deploy_lat', 'deploy_lon', 'end_date', 'end_lat', 'end_lon',
       'drogue_lost_date', 'typedeath', 'lon360', 'err_lat', 'err_lon',
       'err_ve', 'err_vn', 'x', 'y', 'vex', 'vny', 'vxy', 'ae', 'an', 'aen',
       'aex', 'any', 'axy', 'vex_diff', 'vny_diff', 'vxy_diff'],
      dtype='object')

In [31]:
cluster.close()

distributed.scheduler - ERROR - Error transitioning "('assign-6b8ff56bffcc1476f0f1911dbc731039', 453)" from 'processing' to 'erred'
Traceback (most recent call last):
  File "/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/distributed/scheduler.py", line 1809, in _transition
    recommendations, client_msgs, worker_msgs = func(
TypeError: transition_processing_erred() missing 1 required positional argument: 'worker'
distributed.scheduler - ERROR - transition_processing_erred() missing 1 required positional argument: 'worker'
Traceback (most recent call last):
  File "/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/distributed/utils.py", line 742, in wrapper
    return await func(*args, **kwargs)
  File "/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/distributed/scheduler.py", line 4829, in remove_worker
    self.transitions(recommendations, stimulus_id=stimulus_id)
  File "/home1/datahome/mdemol/.miniconda3/